<a href="https://colab.research.google.com/github/nazariosafariesqi/BigData/blob/main/Jobsheet13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BIG DATA JS 13**


In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

In [5]:
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [7]:
!pip install -q findspark

In [8]:
import findspark
findspark.init()

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
 .master("local")\
 .appName("Colab")\
 .config('spark.ui.port', '4050')\
 .getOrCreate()

In [10]:
!wget https://raw.githubusercontent.com/dhanifudin/pyspark-demo/main/sample_books.json

--2022-06-06 15:33:35--  https://raw.githubusercontent.com/dhanifudin/pyspark-demo/main/sample_books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1565 (1.5K) [text/plain]
Saving to: ‘sample_books.json’

sample_books.json   100%[===================>]   1.53K  --.-KB/s    in 0s      

2022-06-06 15:33:35 (22.9 MB/s) - ‘sample_books.json’ saved [1565/1565]



In [11]:
df = spark.read.json("sample_books.json")
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



In [12]:
df.show(4,False)

+---------------+--------------+-----+----------------+------------+
|author         |edition       |price|title           |year_written|
+---------------+--------------+-----+----------------+------------+
|Austen, Jane   |Penguin       |18.2 |Northanger Abbey|1814        |
|Tolstoy, Leo   |Penguin       |12.7 |War and Peace   |1865        |
|Tolstoy, Leo   |Penguin       |13.5 |Anna Karenina   |1875        |
|Woolf, Virginia|Harcourt Brace|25.0 |Mrs. Dalloway   |1925        |
+---------------+--------------+-----+----------------+------------+
only showing top 4 rows



In [13]:
df.select("title", "price", "year_written").show(5)

+----------------+-----+------------+
|           title|price|year_written|
+----------------+-----+------------+
|Northanger Abbey| 18.2|        1814|
|   War and Peace| 12.7|        1865|
|   Anna Karenina| 13.5|        1875|
|   Mrs. Dalloway| 25.0|        1925|
|       The Hours|12.35|        1999|
+----------------+-----+------------+
only showing top 5 rows



In [14]:
df_filtered = df.filter("year_written > 1950 AND price > 10 AND title IS NOT NULL")
df_filtered.select("title", "price", "year_written").show(50,False)

+-----------------------------+-----+------------+
|title                        |price|year_written|
+-----------------------------+-----+------------+
|The Hours                    |12.35|1999        |
|Harry Potter                 |19.95|2000        |
|One Hundred Years of Solitude|14.0 |1967        |
+-----------------------------+-----+------------+



In [15]:
from pyspark.sql.functions import mean, min, max
result = df.select([max("price")])
result.show()

+----------+
|max(price)|
+----------+
|      29.0|
+----------+



1. Tampilkan data buku dengan harga paling murah! 

In [16]:
from pyspark.sql.functions import mean, min, max
result = df.select([min("price")])
result.show()

+----------+
|min(price)|
+----------+
|      5.75|
+----------+



2. Tampilkan jumlah terbit buku dikategorikan setiap tahun ditulis! 


In [17]:
from pyspark.sql.functions import mean, min, max, count
df.groupBy("title", "year_written").count().show()

+--------------------+------------+-----+
|               title|year_written|count|
+--------------------+------------+-----+
|        Harry Potter|        2000|    1|
|    Huckleberry Finn|        1865|    1|
|Hamlet, Prince of...|        1603|    1|
|         Bleak House|        1870|    1|
|       War and Peace|        1865|    1|
| A Room of One's Own|        1922|    1|
|   Lord of the Rings|        1937|    1|
|       Anna Karenina|        1875|    1|
|       Mrs. Dalloway|        1925|    1|
|           The Hours|        1999|    1|
|One Hundred Years...|        1967|    1|
|          Tom Sawyer|        1862|    1|
|    Northanger Abbey|        1814|    1|
+--------------------+------------+-----+



3. Tampilkan data buku termahal tiap tahun penulisannya! 

In [18]:
from pyspark.sql.functions import mean, min, max, count
df.groupBy("title", "price").max().show()

+--------------------+-----+----------+-----------------+
|               title|price|max(price)|max(year_written)|
+--------------------+-----+----------+-----------------+
|       Anna Karenina| 13.5|      13.5|             1875|
|   Lord of the Rings|27.45|     27.45|             1937|
|       Mrs. Dalloway| 25.0|      25.0|             1925|
|    Northanger Abbey| 18.2|      18.2|             1814|
|    Huckleberry Finn| 5.76|      5.76|             1865|
|         Bleak House| 5.75|      5.75|             1870|
|          Tom Sawyer| 7.75|      7.75|             1862|
|           The Hours|12.35|     12.35|             1999|
|One Hundred Years...| 14.0|      14.0|             1967|
|       War and Peace| 12.7|      12.7|             1865|
|Hamlet, Prince of...| 7.95|      7.95|             1603|
| A Room of One's Own| 29.0|      29.0|             1922|
|        Harry Potter|19.95|     19.95|             2000|
+--------------------+-----+----------+-----------------+



4. Tampilkan data buku termurah tiap tahun penulisannya!

In [19]:
from pyspark.sql.functions import mean, min, max, count
df.groupBy("title", "price").min().show()

+--------------------+-----+----------+-----------------+
|               title|price|min(price)|min(year_written)|
+--------------------+-----+----------+-----------------+
|       Anna Karenina| 13.5|      13.5|             1875|
|   Lord of the Rings|27.45|     27.45|             1937|
|       Mrs. Dalloway| 25.0|      25.0|             1925|
|    Northanger Abbey| 18.2|      18.2|             1814|
|    Huckleberry Finn| 5.76|      5.76|             1865|
|         Bleak House| 5.75|      5.75|             1870|
|          Tom Sawyer| 7.75|      7.75|             1862|
|           The Hours|12.35|     12.35|             1999|
|One Hundred Years...| 14.0|      14.0|             1967|
|       War and Peace| 12.7|      12.7|             1865|
|Hamlet, Prince of...| 7.95|      7.95|             1603|
| A Room of One's Own| 29.0|      29.0|             1922|
|        Harry Potter|19.95|     19.95|             2000|
+--------------------+-----+----------+-----------------+

